In [ ]:
import os
os.chdir('../')
%pwd

: 

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    weight_decay_rate: float
    learning_rate: float
    batch_size: int


: 

In [ ]:
from textSummerizer.constants import *
from textSummerizer.utils import read_yaml, create_directories

: 

In [ ]:
class configurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            model_ckpt= config.model_ckpt,
            num_train_epochs= params.NUM_TRAIN_EPOCHS,
            learning_rate= params.LEARNING_RATE,
            weight_decay_rate= params.WEIGHT_DECAY_RATE,
            batch_size= params.BATCH_SIZE
            
        )

        return model_trainer_config

: 

In [ ]:
from transformers import AdamWeightDecay
from transformers import DataCollatorForSeq2Seq
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk
import tensorflow as tf

: 

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):        
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_t5_small = TFAutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_t5_small, return_tensors='tf')
        
        #loading data 
        dataset_samsum_tf = load_from_disk(self.config.data_path)

        train_dataset = dataset_samsum_tf["train"].to_tf_dataset(
             batch_size= self.config.batch_size,
             columns=["input_ids", "attention_mask", "labels"],
             shuffle=True,
            collate_fn=seq2seq_data_collator,
            )
        
        validation_dataset = dataset_samsum_tf["validation"].to_tf_dataset(
             batch_size= self.config.batch_size,
             columns=["input_ids", "attention_mask", "labels"],
             shuffle=False,
             collate_fn=seq2seq_data_collator
             )      


        optimizer = AdamWeightDecay(learning_rate= self.config.learning_rate, 
                                    weight_decay_rate= self.config.weight_decay_rate
                                    )
        model_t5_small.compile(optimizer= optimizer)

        model_t5_small.fit(
            train_dataset,
            validation_data=validation_dataset,
            epochs= self.config.num_train_epochs)

        ## Save model
        model_t5_small.save_pretrained(os.path.join(self.config.root_dir,"t5-small-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

: 

In [ ]:
import sys
from textSummerizer.exception import CustomException

: 

In [ ]:
try:
    config = configurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise CustomException(e, sys)

: 